# COLX 531 Lab 4 T4 Team 7 Transformer + FastText



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 24kB/s 
     |████████████████████████████████| 5.9MB 59.6MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu101 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [3]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git
!cd OpenNMT-py; pip install -e .

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17272, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 17272 (delta 139), reused 103 (delta 67), pack-reused 17044
Receiving objects: 100% (17272/17272), 273.37 MiB | 29.45 MiB/s, done.
Resolving deltas: 100% (12439/12439), done.
Obtaining file:///content/OpenNMT-py
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 645kB 22.4MB/s 
     |████████████████████████████████| 14.3MB 18.0MB/s 
     |████████████████████████████████| 1.2MB 50.6MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=ef8787b03a30481dd5c62be6de059727ef17411a00631e71348cf5afc7b529d7
  Stored in directory: /root/.cache/pip/wheels/d6/61/f

In [4]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pt.300.vec.gz
!gzip -dk cc.en.300.vec.gz
!gzip -dk cc.pt.300.vec.gz

--2021-04-20 23:00:14--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  12.2MB/s    in 1m 45s  

2021-04-20 23:02:00 (12.0 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2021-04-20 23:02:00--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pt.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1271093660 (1.2G) [binary/octet-stream]
Saving to: ‘cc.pt.

In [5]:
!python -m spacy download pt_core_news_sm
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 21.2MB 7.1MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp37-none-any.whl size=21186283 sha256=e8193d593b9bff0cb50df8c3896fd788e51f0ee534311f9381f20d6967e42262
  Stored in directory: /tmp/pip-ephem-wheel-cache-xo55j8o4/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [6]:
import pt_core_news_sm
import en_core_web_sm

spacy_pt = pt_core_news_sm.load()
spacy_en = en_core_web_sm.load()

In [ ]:
EN_FILE_LIST = ['./drive/MyDrive/585/train.en', './drive/MyDrive/585/valid.en']
PT_FILE_LIST = ['./drive/MyDrive/585/train.pt', './drive/MyDrive/585/valid.pt']

In [ ]:
for file in EN_FILE_LIST:
  with open(file,"r", encoding='utf-8') as fp:
    outfile = file + ".tokd"
    with open(outfile, "w", encoding="utf-8") as out:
      for line in fp:
        tokenized = [tok.text for tok in spacy_en.tokenizer(line)]
        out.write(" ".join(tokenized))
      out.close()
      fp.close()

In [ ]:
for file in PT_FILE_LIST:
  with open(file,"r", encoding='utf-8') as fp:
    outfile = file + ".tokd"
    with open(outfile, "w", encoding="utf-8") as out:
      for line in fp:
        tokenized = [tok.text for tok in spacy_pt.tokenizer(line)]
        out.write(" ".join(tokenized))
      out.close()
      fp.close()

## 1. Build the vocab

In [ ]:
!onmt_build_vocab -config ./drive/MyDrive/585/transformer-fasttext-10k-vocab.yml -n_sample 10000
# Danish spaCy only src:18253
# Danish spaCy + sentencepiece src: 17810
# Danish + Norwegian spaCy + sentencepiece src: 20350
# English spaCy only tgt:14772
# English spaCy + sentencepiece tgt: 14454
# English spaCy + sentencepiece tgt: 17089 (when Norwegian corpus is present)

# FastText (10000) src:18253 tgt:14772

[2021-04-17 02:04:22,728 INFO] Counter vocab from 10000 samples.
[2021-04-17 02:04:22,728 INFO] Build vocab on 10000 transformed examples/corpus.
[2021-04-17 02:04:22,741 INFO] corpus_1's transforms: TransformPipe(FilterTooLongTransform(src_seq_length=200, tgt_seq_length=200))
[2021-04-17 02:04:22,742 INFO] Loading ParallelCorpus(./drive/MyDrive/585/train.pt.tokd, ./drive/MyDrive/585/train.en.tokd, align=None)...
[2021-04-17 02:04:22,751 WARNING] Empty line exists in corpus_1#379.
[2021-04-17 02:04:22,752 WARNING] Empty line exists in corpus_1#418.
[2021-04-17 02:04:22,753 WARNING] Empty line exists in corpus_1#464.
[2021-04-17 02:04:22,755 WARNING] Empty line exists in corpus_1#587.
[2021-04-17 02:04:22,755 WARNING] Empty line exists in corpus_1#589.
[2021-04-17 02:04:22,763 WARNING] Empty line exists in corpus_1#938.
[2021-04-17 02:04:22,764 WARNING] Empty line exists in corpus_1#946.
[2021-04-17 02:04:22,764 WARNING] Empty line exists in corpus_1#984.
[2021-04-17 02:04:22,772 WARNIN

## 2. Train Model

In [ ]:
# Freeze word embeddings; 10k vocab + lr 2.0
!onmt_train -config ./drive/MyDrive/585/transformer-fasttext-10k-vocab.yml
# * enc: 16068 match, 2187 missing, (88.02%) 
# * dec: 13767 match, 1009 missing, (93.17%)

Streaming output truncated to the last 5000 lines.
[2021-04-17 02:09:33,316 WARNING] Empty line exists in corpus_1#28769.
[2021-04-17 02:09:33,317 WARNING] Empty line exists in corpus_1#28783.
[2021-04-17 02:09:33,318 WARNING] Empty line exists in corpus_1#28901.
[2021-04-17 02:09:39,507 INFO] Step 300/10000; acc:  29.05; ppl: 42.95; xent: 3.76; lr: 0.00064; 6871/7109 tok/s;    246 sec
[2021-04-17 02:09:41,255 WARNING] Empty line exists in corpus_1#29400.
[2021-04-17 02:09:41,257 WARNING] Empty line exists in corpus_1#29507.
[2021-04-17 02:09:41,257 WARNING] Empty line exists in corpus_1#29516.
[2021-04-17 02:09:41,257 WARNING] Empty line exists in corpus_1#29555.
[2021-04-17 02:09:41,258 WARNING] Empty line exists in corpus_1#29613.
[2021-04-17 02:09:41,258 WARNING] Empty line exists in corpus_1#29616.
[2021-04-17 02:09:41,259 WARNING] Empty line exists in corpus_1#29716.
[2021-04-17 02:09:41,260 WARNING] Empty line exists in corpus_1#29848.
[2021-04-17 02:09:41,263 WARNING] Empty lin

## 3. Decoding/Translate

In [ ]:
# predict
!onmt_translate -model ./drive/MyDrive/585/freeze_10k_lr2_step_3000.pt -src ./drive/MyDrive/585/valid.pt -output ./drive/MyDrive/585/fasttext_dev_3000.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2

[2021-04-17 02:47:57,630 INFO] Translating shard 0.
[2021-04-17 02:48:39,990 INFO] PRED AVG SCORE: -1.0273, PRED PPL: 2.7936


In [11]:
for file in ['./drive/MyDrive/585/pt_preds.txt']:
  with open(file,"r", encoding='utf-8') as fp:
    outfile = file + ".tokd"
    with open(outfile, "w", encoding="utf-8") as out:
      for line in fp:
        tokenized = [tok.text for tok in spacy_pt.tokenizer(line)]
        out.write(" ".join(tokenized))
      out.close()
      fp.close()

In [8]:
# predict
!onmt_translate -model ./drive/MyDrive/585/freeze_10k_lr2_step_3000.pt -src ./drive/MyDrive/585/pt_preds.txt -output ./drive/MyDrive/585/pt_gold_pred.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2

[2021-04-20 23:16:04,248 INFO] Translating shard 0.
[2021-04-20 23:18:08,337 INFO] PRED AVG SCORE: -1.0780, PRED PPL: 2.9387


In [10]:
!perl  OpenNMT-py/tools/multi-bleu.perl ./drive/MyDrive/585/pt2enGold.txt < ./drive/MyDrive/585/pt_gold_pred.txt

BLEU = 8.18, 30.5/12.6/5.2/2.2 (BP=1.000, ratio=1.324, hyp_len=46540, ref_len=35156)


In [12]:
!onmt_translate -model ./drive/MyDrive/585/freeze_10k_lr2_step_3000.pt -src ./drive/MyDrive/585/pt_preds.txt.tokd -output ./drive/MyDrive/585/pt_gold_pred.txt.tokd -gpu 0 -beam_size 5 -seed 531 -block_ngram 2

[2021-04-20 23:19:48,617 INFO] Translating shard 0.
[2021-04-20 23:21:40,001 INFO] PRED AVG SCORE: -1.0475, PRED PPL: 2.8505


In [13]:
!perl  OpenNMT-py/tools/multi-bleu.perl ./drive/MyDrive/585/pt2enGold.txt < ./drive/MyDrive/585/pt_gold_pred.txt.tokd

BLEU = 7.31, 29.0/11.4/4.5/1.9 (BP=1.000, ratio=1.324, hyp_len=46563, ref_len=35156)


In [ ]:
# bleu
!perl  OpenNMT-py/tools/multi-bleu.perl ./drive/MyDrive/585/valid.en < ./drive/MyDrive/585/fasttext_dev_3000.txt

BLEU = 9.05, 34.7/13.4/5.7/2.6 (BP=1.000, ratio=1.042, hyp_len=14523, ref_len=13937)


In [ ]:
# bleu
!perl  OpenNMT-py/tools/multi-bleu.perl ./drive/MyDrive/585/pt_gold.txt < ./drive/MyDrive/585/pt_preds.txt

BLEU = 58.39, 75.3/63.7/54.1/46.0 (BP=0.993, ratio=0.994, hyp_len=35044, ref_len=35273)
